In [ ]:
#run only once
%pip install openai chromadb pandas numpy scikit-learn python-dotenv

In [2]:
# ── Cell 2: Imports ─────────────────────────────────────────
import sys
sys.path.append('..')

from src.data_loader  import load_episodes
from src.get_embeddings import get_embeddings_batches
from src.vector_store import build_collection, get_collection
from src.search       import semantic_search
from src.llm_integeration import interpret_emotional_query
from src.memory       import ConversationMemory, Turn
from src.rag_pipeline import run_pipeline, print_results

print('✓ All imports OK')

✓ All imports OK


In [ ]:
# ── Cell 0A: Install (once) ──────────────────────────────────
# %pip install youtube-transcript-api

# ── Cell 0B: Fetch all transcripts ──────────────────────────
import sys
sys.path.append('..')

from src.transcript_fetcher import fetch_all_transcripts, print_summary

# Reads URLs from data/youtube_urls.txt
# Saves results to data/processed/transcripts_df.csv
# Logs errors to logs/transcript_errors.log
df = fetch_all_transcripts(sleep_secs=1.0)

print_summary(df)

# ── Cell 0C: Pass URLs directly (alternative) ────────────────
my_urls = [
    "https://www.youtube.com/watch?v=TbsRU-crgsc",
    "https://www.youtube.com/watch?v=jroF3PH-PTs",
]

df = fetch_all_transcripts(urls=my_urls)
print_summary(df)



In [3]:
# ── Cell 3: Load data with embeddings ────────────────────────────────────────
df = load_episodes()
df[['youtube_title', 'youtube_channel', 'duration_mins', 'word_count']].head()

✓ embedding type: <class 'list'> <class 'float'>
✓ Loaded 20 episodes


,youtube_title,youtube_channel,duration_mins,word_count
0,"This Is Why You FEEL LOST, LAZY & UNMOTIVATED ...",Lewis Howes,59.5,10870
1,Brené Brown: The Algorithms Have Forced Us Int...,The Diary Of A CEO,110.7,19185
2,Brené Brown — How to Save Your Marriage,Tim Ferriss,79.4,13602
3,The Science of Making & Breaking Habits: How t...,Mel Robbins,86.3,19153
4,Why You Feel Lost in Life: Dr. Gabor Maté on T...,Mel Robbins,77.9,12425


In [4]:
df['embedding'].head()

0    [0.06991995871067047, 0.02282216027379036, 0.0...
1    [0.02198505401611328, 0.01518611703068018, -0....
2    [0.04754159599542618, 0.02921060100197792, -0....
3    [0.024050140753388405, 0.030886655673384666, -...
4    [0.04396381974220276, 0.03935940936207771, -0....
Name: embedding, dtype: object

In [6]:
# ── Cell 4: Build ChromaDB (RUN ONCE ONLY) ──────────────────
collection = build_collection(df, force_rebuild=True)
print(f'Collection has {collection.count()} episodes')

deleted old collection
created collection: podcast_episodes
✅ Added 20 episodes to ChromaDB
Collection has 20 episodes


In [7]:
# ── Cell 5: Load existing collection (USE EVERY SESSION) ────
collection = get_collection()

✓ Loaded 'podcast_episodes' (20 episodes)


In [8]:
# ── Cell 6: Test search ──────────────────────────────────────
test_queries = [
    'I feel like a failure after not getting that job',
    'I am angry and I do not know why',
    'I want to love myself more',
]

for q in test_queries:
    print(f"\n🔍 '{q}'")
    results = semantic_search(q, collection, top_k=2)
    for r in results:
        print(f"  → {r['metadata']['episode_title'][:55]}  ({r['similarity']:.2f})")


🔍 'I feel like a failure after not getting that job'
  → Brené Brown: The Algorithms Have Forced Us Into A Hidde  (1.00)
  → Brené Brown: The Algorithms Have Forced Us Into A Hidde  (0.00)

🔍 'I am angry and I do not know why'
  → Brené Brown — How to Save Your Marriage  (1.00)
  → Repairing Self-Abandonment: People Pleasing, Anxious At  (0.00)

🔍 'I want to love myself more'
  → How Do I Learn to Love Myself, Really? | The Mel Robbin  (1.00)
  → Brené Brown — How to Save Your Marriage  (0.00)


In [ ]:
# ── Cell 7: Full RAG pipeline ────────────────────────────────
memory     = ConversationMemory()
user_query = 'I feel really stressed about finding a new job and keep doubting myself'

output = run_pipeline(
    user_query = user_query,
    collection = collection,
    memory     = memory,
    top_k      = 3,
)

print_results(output)

In [ ]:
# ── Cell 8: Check memory ─────────────────────────────────────
print(f'Memory has {len(memory)} turn(s)\n')
print(memory.build_context_string())

In [ ]:
# ── Cell 9: Second turn (memory feeds into LLM) ─────────────
output2 = run_pipeline(
    user_query = 'Now I feel ashamed that I am not handling this better',
    collection = collection,
    memory     = memory,
    top_k      = 2,
)

print_results(output2)
print(f'\nMemory now has {len(memory)} turns')

In [ ]:
# ── Cell 10: Reset memory ────────────────────────────────────
memory.clear()
print(f'Cleared. Turns: {len(memory)}')